#### Preprocess data

In [ ]:
#!od3d platform run -p torque -c "od3d dataset setup -d co3dv1_zsp_10s"

In [ ]:
#!od3d platform run -p torque -c "od3d dataset extract-meta -d co3dv1_zsp_10s"

In [ ]:
#!od3d platform run -p torque -c "od3d dataset preprocess -d co3dv1_zsp_10s"


In [1]:
from typing import List, Union
from od3d.datasets.co3dv1 import CO3Dv1
from od3d.cv.visual.show import  show_scene2d , show_scene ,show_pcl_via_open3d
from od3d.datasets.co3dv1.dataset import CO3Dv1_Sequence
from od3d.cv.geometry.transform import transf3d_broadcast
from od3d.cv.cluster.embed import tsne, pca 
import torch
import matplotlib
import open3d as o3d

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
def get_sequence_data(sequences: List[CO3Dv1_Sequence] , index: int):
    sequence = sequences[index]
    mesh_verts = sequence.get_mesh().verts_ncds.clone()
    cams_tform4x4_world, cams_intr4x4, cams_imgs = sequence.get_cams(cams_count=-1)
    #mesh.verts = transf3d_broadcast(mesh.verts),#sequence.zsp_labeled_cuboid_ref_tform_obj)
    sequence_dict = {
        'feats': sequence.get_mesh_feats(),
        'viewpoints': sequence.get_mesh_feats_viewpoint(),
        'cams_intr4x4': cams_intr4x4,
        'cams_tform4x4_world': cams_tform4x4_world,
        'pts3d': mesh_verts ,
        'imgs': cams_imgs}
    return sequence_dict

In [3]:
def show_scene2d(
        pts2d: Union[torch.Tensor, List[torch.Tensor]]=None,
        pts2d_names: List[str]=None,
        pts2d_colors: Union[torch.Tensor, List]=None,
        pts2d_length: int=0,
):
    """
    Args:
        pts2d (Union[torch.Tensor, List[torch.Tensor]]): PxNx2 or List(Npx2)
        pts2d_names (List[str]): (P,)
        pts2d_colors (Union[torch.Tensor, List]): Px2x3 or List(3)
    Returns:
    """
    
    
    

    # scatter plot 2d with legend and colors
    import matplotlib.pyplot as plt
    import matplotlib
    matplotlib.use("TkAgg")

    # scatter plot 2d with legend and colors
    fig, ax = plt.subplots(len(pts2d), 1)
    if pts2d is not None:
        for i, pts2d_i in enumerate(pts2d):
            if pts2d_colors is not None:
                pts2d_colors_i = pts2d_colors[i]
            #else:
              #pts2d_colors_i = get_colors(len(pts2d))[i]
            

            
            if pts2d_length > 0:
                colours = pts2d_colors_i.detach().cpu().numpy()
                ax[i].scatter(pts2d_i[:pts2d_length, 0].detach().cpu().numpy(), pts2d_i[:pts2d_length, 1].detach().cpu().numpy(),
                               c=colours[:pts2d_length], marker = 'x')
                ax[i].scatter(pts2d_i[pts2d_length:, 0].detach().cpu().numpy(), pts2d_i[pts2d_length:, 1].detach().cpu().numpy(),
                               c=colours[pts2d_length:], marker = 'o')
            else:
                ax[i].scatter(pts2d_i[:, 0].detach().cpu().numpy(), pts2d_i[:, 1].detach().cpu().numpy(), c=pts2d_colors_i.detach().cpu().numpy())
            if pts2d_names is not None:
                ax[i].title.set_text(pts2d_names[i])
    plt.show()

In [4]:
sequence_list = [0,1,2]
merge = True
co3d = CO3Dv1.create_by_name('co3dv1_zsp_10s', config={'preprocess' : {'tform_obj': {'enabled': False}},'mesh_feats' :{'enabled':False},'mesh_feats_dist' :{'enabled':False}}) #, config={'categories': [category], 'aligned_name': aligned_name, 'sequences_count_max_per_category': sequences_count}) # co3d_no_zsp_20s_aligned #co3d_5s_no_zsp_labeled 'co3d_50s_no_zsp_aligned' 'co3dv1_10s_zsp_aligned' 'co3d_10s_zsp_aligned' 'co3dv1_10s_zsp_unlabeled'
categories = co3d.categories
sequences = co3d.get_sequences()

seq1 = get_sequence_data(sequences,0) #
seq2 = get_sequence_data(sequences,1) 

if merge:
    seq_feats = seq1["feats"] + seq2["feats"]
print('categories:', categories)

/home/heisenberg/Thesis_repo/od3d/venv_od3d/lib/python3.8/site-packages/hydra/_internal/defaults_list.py:251: UserWarning: In 'defaults': Defaults list is missing `_self_`. See https://hydra.cc/docs/1.2/upgrades/1.0_to_1.1/default_composition_order for more information
  warnings.warn(msg, UserWarning)


categories: ['bicycle', 'toytruck', 'toytrain', 'teddybear', 'car', 'toybus', 'motorcycle', 'keyboard', 'handbag', 'remote', 'toyplane', 'toilet', 'hairdryer', 'mouse', 'toaster', 'hydrant', 'chair', 'laptop', 'book', 'backpack']


In [5]:
seq1_feats = seq1["feats"]
seq2_feats = seq2["feats"]

In [6]:
seq_feats_padded = torch.nn.utils.rnn.pad_sequence(seq_feats , batch_first=True, padding_value=torch.nan)
seq_feats_padded_mask = (~seq_feats_padded.isnan().all(dim=-1))

### PCA 2D and 3D

In [7]:
seq_feats_embed = pca(seq_feats_padded[seq_feats_padded_mask], C=2)
seq_feats_embed = seq_feats_embed.detach().cpu()
seq_feats_embed_3d = pca(seq_feats_padded[seq_feats_padded_mask], C=3)
seq_feats_embed_3d = seq_feats_embed_3d.detach().cpu()

### TSNE 2D and 3D

In [ ]:
seq_feats_embed = tsne(seq_feats_padded[seq_feats_padded_mask], C=2)
seq_feats_embed = seq_feats_embed.detach().cpu()
seq_feats_embed_3d = tsne(seq_feats_padded[seq_feats_padded_mask], C=3)
seq_feats_embed_3d = seq_feats_embed_3d.detach().cpu()

In [8]:
seq_colors_padded = torch.zeros(size=seq_feats_padded.shape[:-1] + (4,), dtype=torch.float32)
seq_colors_padded[:, :, 3] = 0.2

In [9]:

seq_colors_padded_verts = seq_colors_padded.clone()
if merge:
    seq_colors_padded_verts[:len(seq1_feats), :, :3] = seq1['pts3d'][:, None, :3]
    seq_colors_padded_verts[len(seq1_feats):, :, :3] = seq2['pts3d'][:, None, :3]


In [10]:
if merge:
    seq_feats_viewpoints = seq1['viewpoints'] + seq2['viewpoints']
else:
    seq_feats_viewpoints = seq1['viewpoints']

seq_feats_viewpoints_padded = torch.nn.utils.rnn.pad_sequence(seq_feats_viewpoints , batch_first=True,
                                                     padding_value=torch.nan)
seq_feats_viewpoints_padded = (seq_feats_viewpoints_padded - seq_feats_viewpoints_padded[seq_feats_padded_mask].min(dim=0).values[None, None]) \
                                / (seq_feats_viewpoints_padded[seq_feats_padded_mask].max(dim=0).values[None, None] - seq_feats_viewpoints_padded[seq_feats_padded_mask].min(dim=0).values[None, None])

seq1_colors_padded_viewpoint = seq_colors_padded.clone()
if merge:
    seq1_colors_padded_viewpoint[:, :, :3] = seq_feats_viewpoints_padded[:, :, :3]

In [11]:
seq1_embed_length =sum([seq1_feats[i].shape[0] for i in range(len(seq1_feats))])
seq1_embed_length

9484

In [12]:
show_scene2d(pts2d=[seq_feats_embed[:], seq_feats_embed[:]],
             pts2d_colors=[seq_colors_padded_verts[seq_feats_padded_mask], seq1_colors_padded_viewpoint[seq_feats_padded_mask]],
             pts2d_length= seq1_embed_length)

In [15]:
colours = seq_colors_padded_verts[seq_feats_padded_mask].numpy()[:,:3]
colours
seq_feats_embed_3d.shape

torch.Size([18822, 3])

In [16]:
show_scene(pts3d=seq_feats_embed_3d[:], pts3d_colors=colours)
             #pts3d_colors=[seq_colors_padded_verts[seq_feats_padded_mask], seq1_colors_padded_viewpoint[seq_feats_padded_mask]])

Backend TkAgg is interactive backend. Turning interactive mode on.


RuntimeError: Unable to cast Python instance of type <class 'numpy.float32'> to C++ type '?' (#define PYBIND11_DETAILED_ERROR_MESSAGES or compile in debug mode for details)